In [3]:
from vedo import *
from utils import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import pickle
import matplotlib.pyplot as plt

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------
np.set_printoptions(precision=8, linewidth = 75)

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


# Pose Graph Optimization

### GOAL: Adjust configuration of absolute poses (nodes) to minimize squared error intorduced by constraints (edges)


Node: [x, y, z, r, p, y]

Edge: Odometry Measurements

# State vector $x$

absolute position and orientation of vehicle at each timestep

\begin{equation}
\textbf{x} = 
\begin{bmatrix}
\textbf{x}_1\\
\textbf{x}_2\\
\textbf{x}_3 \\
\vdots
\end{bmatrix} = 
\begin{bmatrix}
x_1 & y_1 & z_1 & r_1 & p_1 & y_1 \\
x_2 & y_2 & z_2 & r_2 & p_2 & y_2 \\
x_3 & y_3 & z_3 & r_3 & p_3 & y_3 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \\ 
\end{bmatrix}
\end{equation}

# Transformations $\mathbf{X}$  
Represeneted in Homogenous Coordinates. This is an overparameterization (similar to using quaternions) which helps avoid singularities when calculating the error function to adjust each Gauss-Newton iteration, however, the additional degrees of freedom need to be removed before forming our state vector

\begin{equation}
\mathbf{X_i} = 
\begin{bmatrix}
R_{11} & R_{12} & R_{13} & dx\\
R_{21} & R_{22} & R_{23} & dy \\
R_{31} & R_{32} & R_{33} & dz \\
0 & 0 & 0 & 1
\end{bmatrix} 
\end{equation}


$\big(\mathbf{X}_i^{-1} \mathbf{X}_{i+1} \big)$  describes how node $i$ sees node $(i+1)$ (ex: odometry)

$\big(\mathbf{X}_i^{-1} \mathbf{X}_{j} \big)$  describes how node $i$ sees node $j$ (ex: loop closure)


<span style="color:red"> Important note: $X$ will need to contain more elements than $x$, since $X$ must represent all state pairs observed in constraints $Z$, while $x$ only needs a single element to represent the absolute pose of each state </span>


In [ ]:
def v2t(vector):
    """converts a transformation vector to homogenous coordinate system"""
    if len(tf.shape(vector)) == 1: #allow for 1-D or N-D input
        vector = vector[None,:]
    angs = vector[:,3:]
    phi = angs[:,0]
    theta = angs[:,1]
    psi = angs[:,2]
    rot = tf.Variable([[cos(theta)*cos(psi), sin(psi)*cos(phi) + sin(phi)*sin(theta)*cos(psi), sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi)],
                       [-sin(psi)*cos(theta), cos(phi)*cos(psi) - sin(phi)*sin(theta)*sin(psi), sin(phi)*cos(psi) + sin(theta)*sin(psi)*cos(phi)],
                       [sin(theta), -sin(phi)*cos(theta), cos(phi)*cos(theta)]])
    rot = tf.transpose(rot, [2, 0, 1])
    trans = vector[:,:3]
    trans = np.reshape(trans, (np.shape(rot)[0], 3, 1))
    transform = tf.concat((rot, trans), axis = -1)
    extra = tf.tile(tf.constant([[[0., 0., 0., 1.]]], dtype = tf.double), (np.shape(rot)[0],1,1))
    transform = tf.concat((transform, extra), axis = -2)
    return transform

In [ ]:
def t2v(mat):
    """converts transformation matrix to state vector"""
    if len( tf.shape(mat) ) == 2:
        mat = mat[None, :, :]
    R_sum = np.sqrt(( mat[:,0,0]**2 + mat[:,0,1]**2 + mat[:,1,2]**2 + mat[:,2,2]**2 ) / 2)
    phi = np.arctan2(-mat[:,1,2],mat[:,2,2])
    theta = np.arctan2(mat[:,0,2], R_sum)
    psi = np.arctan2(-mat[:,0,1], mat[:,0,0])
    angs = np.array([phi, theta, psi])
    vector = tf.concat((mat[:,:3,-1], angs.T), axis =1)
    return vector

In [ ]:
#NOTE: v2t() -> t2v() is returning to the same value only if I take inverse (for rotation) 
#                     and without inverse only (for translation). [T]^-1 @ [T] = I still
#Question: is this normal?  

# # test = np.array([1., 2., 3., 0.05, 0.00, -0.14])
# test = np.ones([2,6])
# test[1,:] = np.array([1., 2., 3., -0.003, 0.001, -0.3])
# T = v2t(test)
# # print("T: \n",T)
# vect_trans = t2v(T).numpy()[:,:3]
# vect_rot = t2v(tf.linalg.inv(T)).numpy()[:,3:]
# print(np.append(vect_trans, vect_rot, axis = 1))

# Least Squares Error Function


The optimial state vector, $x^*$, occurs where:

\begin{equation}
\Large
x^* = \arg\min_x \sum_{ij}^{} e^T_{ij}(x_i,x_j)\Omega_{ij}e_{ij}(x_i,x_j)
\end{equation}

$\Omega_{ij}$ is the information matrix associated with the odometry estimate that relates $i$ and $j$. $\Omega_{ij}$ is the inverse of the covariance matrix $\sigma_{ij}$

the error funcion for each connected node $i,j$ as a function of the state vector $x$ is defined as:

\begin{equation}
\Large
e_{ij}(x) = \text{t2v}(Z_{ij}^{-1} (X_j^{-1}X_i))
\end{equation}

\begin{equation}
Z_{ij}^{-1} = \text{constraint (from measurement)}
\end{equation}

\begin{equation}
(X_j^{-1}X_i) = x_i \text{ relative to }x_j \text{ given the current model of system} 
\end{equation}

$ \text{Stachniss and Grisetti use} \boxplus \text{to repersent the mapping of euclidian space } v \text{ to a homogenous coordinate frame } t$


Important Note: $e_{ij}(x)$ only depends on $x_i$ and $x_j$ 

In [ ]:
Zij = v2t(np.array([[1.,  1.9, 0., 0.000, 0.000, 0.00 ]])) # get from graph 
Xij = v2t(np.array([[1.1, 2.0, 0., 0.,     0.0,    0.00]])) # get from odometry message
# Zij =  v2t(np.zeros([2,6]))
# Xij =  v2t(np.random.randn(2,6))
ij = np.array([[0,1],[1,2]])

e = get_e(Zij, Xij)
print(e)

# a, b = get_A_ij_B_ij(e)
# # print(a)

# J = get_J(e, ij)
# # print(J[1])

# Linearizing the System

\begin{equation}
\Large
e_{ij}(x + \Delta x) \approx e_{ij}(x) + J_{ij} \Delta x
\end{equation}

Here, $J_{ij}$ is the the jacobian of $e_{ij}$ with respect to x

\begin{equation}
\Large
J_{ij} = \frac{\delta e_{ij}(x)}{\delta x} = \bigg{(} 0 \dots \frac{\delta e_{ij}(x_i)}{\delta x_i}
\dots \frac{\delta e_{ij}(x_j)}{\delta x_j} \dots 0 \bigg{)} = \bigg{(} 0 \dots A_{ij} \dots B_{ij} \dots 0 \bigg{)} 
\end{equation}

\begin{equation}
A_{ij}, B_{ij} \in \mathbb{R}^{6 \times 6}
\end{equation}

\begin{equation}
J_{ij}(x) \in \mathbb{R}^{6 \times 6N}
\end{equation}
where $N$ is the total number of poses being solved for (nodes)

In [ ]:
def get_J(e, ij):
    """Forms sparse jacobian J, with entries A and B at indices i and j 
    J == [N, 6, 6*N], N = number of nodes
    e == [N, 6] error vectors
    ij == [N, 2] contains scan indices of each node ex: [scan2, scan5]
    """
    total_num_of_nodes = np.max(ij) + 1 #TODO: is this too big??
    if len(tf.shape(ij))< 2: #expand dimensions if only single pair passed in
        ij = ij[None,:]
    A_ij, B_ij = get_A_ij_B_ij(e)
    
    # Need to tile DIFFERENT AMOUNT depending on the index 
    #    TODO: move to batch operation?
    J = tf.zeros([0,6,total_num_of_nodes*6])
    for k in range(len(ij)):
        #TODO: add logic for when i and j are same value, when j = i + 1 ...
        leading = tf.tile(tf.zeros([6,6]), [1, ij[k,0] ]) #leading zeros before i
#         print("\n leading \n", leading)
        between = tf.tile(tf.zeros([6,6]), [1, ij[k,1] - ij[k,0] - 1 ]) #zeros between i and j
#         print("\n between: \n", between)
        ending  = tf.tile(tf.zeros([6,6]), [1, total_num_of_nodes - ij[k,1] - 1 ])
        J_ij = tf.concat([leading, A_ij[k], between, B_ij[k], ending], axis = 1)
        J = tf.concat((J, J_ij[None,:,:]), axis = 0)

    return J

In [ ]:
# # ij = tf.constant(odometry_history[1:3,6:].astype(int))
# ij = np.array([[1,3],
#                [0,2]])
# print(ij)
# # print(np.max(ij))
# # print("e: \n", e)

# J = get_J(e, ij)  
# print("\n J \n",J[-1])
# print("\n J \n",tf.shape(J))  
# # print("\n", np.around(J.numpy(), 3))

# Accumumlate Sparse Tensors

calculate jacobians in batch operation first, then bring to sparse tensor for matmul??

\begin{equation}
\Large
b^T = \sum_{ij}^{}b_{ij} = \sum_{ij}^{}e_{ij}^T \Omega_{ij} J_{ij}
\end{equation}

In [ ]:
def get_information_matrix(pred_stds):
    """returns information matrix (omega) from ICET cov_estimates"""
#     #I think this is wrong ... 
#     pred_stds = tf.convert_to_tensor(pred_stds)[:,:,None] #convert to TF Tensor
#     cov = pred_stds @ tf.transpose(pred_stds, (0,2,1))    #convert predicted stds -> covariance matrix
#     info = tf.linalg.pinv(cov) #invert
    
    #debug - set to identity
    info = tf.tile(tf.eye(6)[None,:,:], [tf.shape(pred_stds)[0] , 1, 1])
    info = tf.cast(info, tf.double)

#     #debug - weigh rotations more heavily than translations
# #     m = tf.linalg.diag(tf.constant([1., 1., 1., 10., 10., 10.]))
# #     m = tf.linalg.diag(tf.constant([10., 10., 10., 1., 1., 1.])) #vice-versa
#     m = tf.linalg.diag(tf.constant([100., 100., 100., 100., 100., 100.])) #huge values
#     info = tf.tile(m[None,:,:], [tf.shape(pred_stds)[0] , 1, 1])
#     info = tf.cast(info, tf.double)


    return info

In [ ]:
test = get_information_matrix(pred_stds_history)
# print(test)

In [ ]:
def get_b(e, omega, J):
    """gets b matrix, using batched operations """
    b_T = tf.math.reduce_sum(tf.transpose(e, (0,2,1)) @ omega @ tf.cast(J, tf.double), axis = 0)
#     print("\n b_T: \n", tf.shape(b_T))
    b = tf.transpose(b_T)
    return b

In [ ]:
def get_ij(ij_raw):
    """generates ij matrix, which describes which nodes are connected to 
       each other through odometry constraints. 
       Removes skipped indices and shifts everything to start at 0"""
#     print("ij_raw: \n", ij_raw)    
    y, idx = tf.unique(tf.reshape(ij_raw, [-1]))
#     print("\n y: \n", y, "\n idx: \n", idx)    
    ij = tf.reshape(idx, [-1,2])
    return ij

\begin{equation}
\Large
H = \sum_{ij}^{}H_{ij} = \sum_{ij}^{}J_{ij}^T \Omega J_{ij}
\end{equation}

Init Hessian once at start of simulation, update in place as new linearizations are added(?)

### Need to apply constraint to first node in kinematic chain

\begin{equation}
\Large
H_{11} \rightarrow H_{11} + I
\end{equation}

Optimization routine can't make sense of only relative measurements. We need to specify the first pose as the origin

In [ ]:
def get_H(J, omega):
    """returns hessian H"""
    H_ij = tf.transpose(J, (0,2,1)) @ tf.cast(omega, tf.float32) @ J
    H = tf.math.reduce_sum(H_ij, axis = 0)
    #produces negative eigenvals if we don't fix first point in chain
#     print("\n eigval H before constraint:\n", tf.linalg.eigvals(H))
    constrain_11 = tf.pad(tf.eye(6), [[0,len(H)-6],[0,len(H)-6]]) #was this
#     constrain_11 = tf.pad(tf.ones([6,6]), [[0,len(H)-6],[0,len(H)-6]]) #test - nope
#     constrain_11 = tf.pad(tf.linalg.diag(tf.constant([1., 0, 0, 0, 0, 0])), [[0,len(H)-6],[0,len(H)-6]]) #test - nope
    H = H + constrain_11
#     print("\n eigval H after constraint:\n", tf.linalg.eigvals(H))
#     H = tf.convert_to_tensor(np.tril(H.numpy()).T + np.tril(H.numpy(),-1)) #force H to be symmetric
    return H

# Convert between $X$ and $x$

\begin{equation}
\Large
X \in \mathbb{R}^{M \times 4 \times 4}
\end{equation}

Where $M$ is the total number of constraints

\begin{equation}
\Large
x \in \mathbb{R}^{N \times 6}
\end{equation}

$N$ is the total number of nodes being optimized for

In [ ]:
def get_X(x, ij):
    """given x, a list of global poses, this function returns 
       the relative transformations X, that describe the same relationships described by the constraints z
       x  -> global poses of each transform
       ij -> indices of first and second element of x being considered
       """
    #get transform of fist elements in each pair, ordered by how they appear in ij
    first_vec = tf.gather(x, ij[:,0])
    second_vec = tf.gather(x, ij[:,1])

    first_tensor = v2t(tf.cast(first_vec, tf.double))
    second_tensor = v2t(tf.cast(second_vec, tf.double))
    #represents pose of x in 2nd node relative to pose in 1st
    X = tf.linalg.pinv(first_tensor) @ second_tensor #was this
#     X = second_tensor @ tf.linalg.pinv(first_tensor) #works better(?)

    return X

# X_test = get_X(x, ij)
# print(X_test)

# Solve the Linear System 

\begin{equation}
\Large
H \Delta x = -b
\end{equation}

In [ ]:
def get_A_ij_B_ij(e_ij):
    """calculates nonzero terms from the Jacobian of error function w.r.t. nodes i and j using TensorFlow
        e_ij == error function [x, y, z, phi, theta, psi]
        
        NOTE: this works with batch operations: error vectors passed in as tensor will result in
                corresponding output in the same order 
    """
    e_ij = tf.cast(e_ij, tf.float32)
    p_point = e_ij[:,:3]
    phi = e_ij[:,3][:,None]
    theta = e_ij[:,4][:,None]
    psi = e_ij[:,5][:,None]
    
    eyes = tf.tile(-tf.eye(3)[None,:,:], [tf.shape(p_point)[0] , 1, 1]) #was this
#     eyes = tf.tile(tf.eye(3)[None,:,:], [tf.shape(p_point)[0] , 1, 1]) #test
    
#     deriv of R() wrt phi
    dRdPhi = tf.Variable([[tf.zeros(len(phi), dtype = phi.dtype)[:,None], (-sin(psi)*sin(phi) + cos(phi)*sin(theta)*cos(psi)), (cos(phi)*sin(psi) + sin(theta)*sin(phi)*cos(psi))],
                       [tf.zeros(len(phi), dtype = phi.dtype)[:,None], (-sin(phi)*cos(psi) - cos(phi)*sin(theta)*sin(psi)), (cos(phi)*cos(psi) - sin(theta)*sin(psi)*sin(phi))], 
                       [tf.zeros(len(phi), dtype = phi.dtype)[:,None], (-cos(phi)*cos(theta)), (-sin(phi)*cos(theta))] ])[:,:,:,0]
    dRdPhi = tf.transpose(dRdPhi, (2,0,1))
    Jx = dRdPhi @ p_point[:,:,None]
    
    # (deriv of R() wrt theta).dot(p_point)
    dRdTheta = tf.Variable([[(-sin(theta)*cos(psi)), (cos(theta)*sin(phi)*cos(psi)), (-cos(theta)*cos(phi)*cos(psi))],
                               [(sin(psi)*sin(theta)), (-cos(theta)*sin(phi)*sin(psi)), (cos(theta)*sin(psi)*cos(phi))],
                               [(cos(theta)), (sin(phi)*sin(theta)), (-sin(theta)*cos(phi))] ])[:,:,:,0]
    dRdTheta = tf.transpose(dRdTheta, (2,0,1))
    Jy = dRdTheta @ p_point[:,:,None]

    # deriv of R() wrt psi
    dRdPsi = tf.Variable([[(-cos(theta)*sin(psi)), (cos(psi)*cos(phi) - sin(phi)*sin(theta)*sin(psi)), (cos(psi)*sin(phi) + sin(theta)*cos(phi)*sin(psi)) ],
                                       [(-cos(psi)*cos(theta)), (-sin(psi)*cos(phi) - sin(phi)*sin(theta)*cos(psi)), (-sin(phi)*sin(psi) + sin(theta)*cos(psi)*cos(phi))],
                                       [tf.zeros(len(phi), dtype = phi.dtype)[:,None],tf.zeros(len(phi), dtype = phi.dtype)[:,None],tf.zeros(len(phi), dtype = phi.dtype)[:,None]]])[:,:,:,0]
    dRdPsi = tf.transpose(dRdPsi, (2,0,1))
    Jz = dRdPsi @ p_point[:,:,None]
    
    top = tf.concat([eyes, Jx, Jy, Jz], axis = 2) #was this
    flipped = tf.transpose(tf.concat([Jx, Jy, Jz], axis = 2), (0,2,1))     #was this
    
    bottom = tf.concat([-flipped, -eyes], axis = 2) #works???
#     bottom = tf.concat([flipped, -eyes], axis = 2) #test

#     top = tf.concat([eyes, tf.zeros(tf.shape(flipped))], axis = 2) #test
#     bottom = tf.concat([tf.zeros(tf.shape(flipped)), -eyes], axis = 2) #test
    
    A_ij = tf.concat([top, bottom], axis = 1) #was this
    B_ij = -A_ij #was this
#     print("\n A_ij: \n", A_ij[0])
    return A_ij, B_ij

In [ ]:
#old
def get_e(Zij, Xij):
    """calculates error function w.r.t. nodes i and j
    Zij == pose j relative to i according to nodes (rotation matrix)
    Xij == pose j relative to i according to constraints (rotation matrix)
    """        
    e = t2v(tf.linalg.pinv(Zij) @ Xij) # was this
#     e = t2v(tf.linalg.pinv(tf.linalg.pinv(Zij) @ Xij)) #nope
#     e = t2v(tf.linalg.pinv(Xij) @ tf.linalg.pinv(Zij) @ Xij) # test - not quite but I think I'm on to something
    
    #get error  in frame of Xij(?)
#     error = tf.linalg.pinv(Zij) @ Xij
#     print("\n error \n", error)
    
    #rotate to align with world frame axis? - no
    
    #rotae to align with frame of Xi
    
#     e = t2v(error)

    return(e)    

In [ ]:
# #NEW
# def get_e(Zij, x, ij):
#     """calculates error function w.r.t. nodes i and j
#     Zij == pose j relative to i according to nodes (rotation matrix)
#     x   == global poses x (vector)
#     ij  == indices of constraints 
#     """        
#     #TODO: return error w.r.t. Xi (rather than wrt global pose)

#     xi = tf.gather(x, ij[:,0])
#     Xi = v2t(tf.cast(xi, tf.double))
    
#     Xij = get_X(x, ij)
    
#     #wrong
#     e = t2v(Xi @ tf.linalg.pinv(Zij) @ Xij)

#     return(e) 

In [401]:
# # test data generated from LeddarTech PixSet trajectory-----------------------------------
# odometry_history = np.load("test_data/leddartech_pixset/T_vec_history.npy")[:15,:]
# pred_stds_history = np.load("test_data/leddartech_pixset/cov_vec_history.npy")[:15,:]
# ij = get_ij(odometry_history[:,6:].astype(np.int32))
# #-----------------------------------------------------------------------------------------

# simplified test data--------------------------------------------------------------------
npts = 3
# odometry_history = np.tile(np.array([0.05, 0.0, 0.02, 0.0, 0.0, 0.15]), (npts,1))
odometry_history = np.tile(np.array([0.2, .10, 0.0, 0.0, 0.0, 0.0]), (npts,1)) #no rotation (for debug)
pred_stds_history = np.tile(np.array([[0.01, 0.01, 0.01, 1e-4, 1e-4, 1e-4]]), (len(odometry_history),1))
ij = np.array([[0,1]])
for i in range(1,len(odometry_history)):
    ij = np.append(ij, np.array([[i, i+1]]), axis = 0)
# ij = np.array([[0,1], [1,2], [2,3], [3,4], [4,5], [5,6], [6,7], [0,7]]) #mess with last constraint
#-----------------------------------------------------------------------------------------

# # random trajectory data------------------------------------------------------------------
# npts = 15
# odometry_history = np.tile(np.array([0.05, 0.0, 0.01, 0.0, 0.0, 0.15]), (npts,1))
# odometry_history += np.array([0.1, 0.1, 0.1, 0.0001, 0.0001, 0.01]) * \
#                     np.random.randn(np.shape(odometry_history)[0], np.shape(odometry_history)[1])
# pred_stds_history = np.tile(np.array([[0.01, 0.01, 0.01, 1e-4, 1e-4, 1e-4]]), (len(odometry_history),1))
# ij = np.array([[0,1]])
# for i in range(1,len(odometry_history)):
#     ij = np.append(ij, np.array([[i, i+1]]), axis = 0)
# # ij = np.array([[0,1], [1,2], [2,3], [3,4], [4,5], [5,6], [6,7], [0,7]]) #mess with last constraint
# #-----------------------------------------------------------------------------------------

# print("odometry history: \n", odometry_history)
# print(pred_stds_history)
# print("\n ij raw: \n", odometry_history[:,-2:])

Z =  v2t(odometry_history[:,:6]) # constraints (from odometry measurements)
X = get_X(x, ij) # relative pose estimates (from initial configuration of graph)
e = get_e(Z, X) #old get_e()
# e = get_e(Z, x, ij) #new get_e()
omega = get_information_matrix(pred_stds_history[:,:6])
# print("\n omega:\n", omega[0])
J = get_J(e, ij)
b = get_b(e[:,:,None], omega, J)
H = get_H(J, omega)
print("ij: \n", ij.T)
# print("e: \n", tf.shape(e))
print("e: \n", tf.shape(e))
print("J: \n", tf.shape(J))
print("omega: \n", tf.shape(omega))
print("b: \n", tf.shape(b))
print("H: \n", tf.shape(H))

ij: 
 [[0 1 2]
 [1 2 3]]
e: 
 tf.Tensor([3 6], shape=(2,), dtype=int32)
J: 
 tf.Tensor([ 3  6 24], shape=(3,), dtype=int32)
omega: 
 tf.Tensor([3 6 6], shape=(3,), dtype=int32)
b: 
 tf.Tensor([24  1], shape=(2,), dtype=int32)
H: 
 tf.Tensor([24 24], shape=(2,), dtype=int32)


In [402]:
# #close estimate for initial conditions 
# x = np.zeros([tf.math.reduce_max(ij)+1,6])
# print(np.shape(x))
# for i in range(1, len(x)):
#     x[i] = x[i-1] + np.array([0.05, 0.0, 0.01, 0., 0., 0.15]) #explodes
# #     x[i] = x[i-1] + np.array([0.05, 0.0, 0.01, 0., 0., 0.0]) #converges to incorrect solution
#     rot = v2t(np.array([0.05, 0.0, 0.01, 0.0, 0.0, 0.15]))[0,:3,:3]
#     x[:,:3] = x[:,:3].dot(rot)
# x = tf.cast(tf.convert_to_tensor(x), tf.float32)
    
# #zero initial conditions
x_init = tf.zeros([tf.math.reduce_max(ij)+1,6])
# print(x)

# #cheat with correct solution
# x_init = tf.convert_to_tensor(x_ground_truth, tf.float32)
# # print(x)

In [403]:
#solve the naive way
np.random.seed(7220)
#get constraints
Z =  v2t(odometry_history[:,:6])   # [N, 6] -> [N, 4, 4]   
x = x_init
print("\n initial global state estimates: \n", np.around(x, 4))    

runlen = 6

for count in range(runlen):
    #create linear system----------------------------
    #update transformation matrix using ground new ground truth solution vector
    X = get_X(x, ij)
    e = get_e(Z, X) #old get_e()
#     e = get_e(Z, x, ij) #new get_e()
    print("\n Z: \n", np.around(t2v(Z), 6))
    print("\n X: \n", np.around(t2v(X), 4))
    print("\n e: \n", np.around(e, 4))
    omega = get_information_matrix(pred_stds_history[:,:6])
    J = get_J(e, ij)
    b = get_b(e[:,:,None], omega, J)
    H = get_H(J, omega)
#     print("\n J[0]: \n", J[0])
    
    #solve linear system-----------------------------
    # H * delta_x = b
    # H^T * H * delta_x = H^T * b
    # delta_x = (H^T * H)^-1 * H^T * b
    delta_x = tf.linalg.pinv(tf.transpose(H) @ H) @ tf.transpose(H) @ tf.cast(-b, tf.float32)
    delta_x = tf.reshape(delta_x, (-1, 6))
#     print("\n delta_x \n", np.around(delta_x, 4))
    
    #update solution vector--------------------------
    x = x + delta_x #was this
    #------------------------------------------------
    
# #debug - force x[0] to zeros
# x = x.numpy()
# x[:,:] -= x[0,:]
# # x[:,:3] -= x[0,:3]
# x = tf.convert_to_tensor(x)
# print(x)
    
print("\n final global state estimates: \n", np.around(x.numpy(), 4))
print("\n x_ground_truth: \n", x_ground_truth)


 initial global state estimates: 
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]

 Z: 
 [[ 0.2  0.1  0.  -0.   0.  -0. ]
 [ 0.2  0.1  0.  -0.   0.  -0. ]
 [ 0.2  0.1  0.  -0.   0.  -0. ]]

 X: 
 [[ 0.  0.  0. -0.  0. -0.]
 [ 0.  0.  0. -0.  0. -0.]
 [ 0.  0.  0. -0.  0. -0.]]

 e: 
 [[-0.2 -0.1  0.  -0.   0.   0. ]
 [-0.2 -0.1  0.  -0.   0.   0. ]
 [-0.2 -0.1  0.  -0.   0.   0. ]]

 Z: 
 [[ 0.2  0.1  0.  -0.   0.  -0. ]
 [ 0.2  0.1  0.  -0.   0.  -0. ]
 [ 0.2  0.1  0.  -0.   0.  -0. ]]

 X: 
 [[ 0.2  0.1 -0.   0.  -0.   0. ]
 [ 0.2  0.1 -0.   0.  -0.   0. ]
 [ 0.2  0.1 -0.   0.  -0.   0. ]]

 e: 
 [[ 0. -0. -0.  0. -0.  0.]
 [ 0. -0. -0.  0. -0.  0.]
 [ 0. -0. -0.  0. -0.  0.]]

 Z: 
 [[ 0.2  0.1  0.  -0.   0.  -0. ]
 [ 0.2  0.1  0.  -0.   0.  -0. ]
 [ 0.2  0.1  0.  -0.   0.  -0. ]]

 X: 
 [[ 0.2  0.1 -0.  -0.   0.  -0. ]
 [ 0.2  0.1  0.  -0.   0.  -0. ]
 [ 0.2  0.1  0.   0.   0.  -0. ]]

 e: 
 [[ 0.  0. -0. -0.  0. -0.]
 [-0.  0.  0. -0.  0. -0.

# Plot Optimized Trajectory

In [404]:
plt1 = Plotter(N = 1, axes = 1, bg =(1, 1, 1), interactive = True) #ax=7 gives rulers
disp = []

#plot estimated solution
c = np.linspace(0.1,1.00, len(x))[:,None]
cname = np.append(-c, c, axis = 1)
cname = np.append(cname, c, axis = 1).tolist()
p = Points(x.numpy()[:,:3], c=cname, r = 10)
disp.append(p)
L = Line(p0=x.numpy()[:,:3], p1=x.numpy()[:,:3], c = 'k', lw = 3).legend("LS Solution") #plot line of estimated trajectory from pose graph
disp.append(L)
disp.append(Line(p0 = x.numpy()[0,:3], p1 = np.array([0,0,0]), lw = 3))#add line between first point in x and origin (to show bug)
o = Points([[0.,0.,0.]], c ='g', r = 15).legend('start position') #plot origin
disp.append(o)
#add in body frame axis for each pose in x
disp = draw_body_frame_axis(x, disp)

#draw ground truth (for debug)
ground_truth_line, x_ground_truth = get_ground_truth(Z, disp)
disp.append(ground_truth_line)
# print("x_estimated: \n", np.around(x.numpy(),4))
# print("\n x_ground_truth: \n", x_ground_truth)

lb = LegendBox([ground_truth_line, L, o], width = 0.25, height = 0.25, markers=["--", '-', '.']).font("Kanopus")
plt1.show(disp, lb, "Graph Slam Test")
ViewInteractiveWidget(plt1.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [ ]:
def get_ground_truth(Z, disp =[]):
    """loops through odometry measurements in Zij and plots absolute poses of each
        FOR DEBUG ONLY- ONLY WORKS WITH LINEAR ODOMETRY HISTORY
    """
    x = np.zeros([len(Z)+1,6])
    Zcum = Z[0]
    for i in range(1, len(Z)+1):
        euls = t2v(Zcum) #convert to euler angles
        x[i] = euls
        testp = Points([[Zcum[0,3], Zcum[1,3], Zcum[2,3]]], r = 7)
        disp.append(testp)
        Zcum = Z[i-1] @ Zcum
#     print("\n x \n", x)
    
    draw_body_frame_axis(tf.convert_to_tensor(x), disp)  
    ground_truth_line = Line(p0=x[:,:3], p1=x[:,:3], lw = 3).pattern('- -', repeats=10).legend("ground truth")
#     disp.append(ground_truth_line)
    return(ground_truth_line, x)

In [142]:
def draw_body_frame_axis(x, disp = []):
    """draws local xyz axis arrows on each pose in x """
    scale = 0.025 #axis length
    for i in range(len(x)):
        rot = R_tf(x[i,3:])
        A = rot @ np.eye(3) * scale
        xvec = Arrow(x.numpy()[i,:3], x.numpy()[i,:3] + A[:,0], c = 'red') 
        yvec = Arrow(x.numpy()[i,:3], x.numpy()[i,:3] + A[:,1], c = 'green') 
        zvec = Arrow(x.numpy()[i,:3], x.numpy()[i,:3] + A[:,2], c = 'blue') 
        disp.append(xvec)
        disp.append(yvec)
        disp.append(zvec)
    return(disp)

In [18]:
def R_tf(angs):
    """generates rotation matrix using euler angles
    angs = tf.constant(phi, theta, psi) (aka rot about (x,y,z))
            can be single set of angles or batch for multiple cells
    """

    if len(tf.shape(angs)) == 1:
        angs = angs[None,:]
#     #old (wrong??)
#     phi = angs[:,0]
#     theta = angs[:,1]
#     psi = angs[:,2]
    #new
    phi = -angs[:,0]
    theta = -angs[:,1]
    psi = -angs[:,2]


    mat = tf.Variable([[cos(theta)*cos(psi), sin(psi)*cos(phi) + sin(phi)*sin(theta)*cos(psi), sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi)],
                       [-sin(psi)*cos(theta), cos(phi)*cos(psi) - sin(phi)*sin(theta)*sin(psi), sin(phi)*cos(psi) + sin(theta)*sin(psi)*cos(phi)],
                       [sin(theta), -sin(phi)*cos(theta), cos(phi)*cos(theta)]
                        ])

    mat = tf.transpose(mat, [2, 0, 1])
    mat = tf.squeeze(mat)
    return mat

# Notes

Takes ~100 iterations to converge for 10 samples

zero initial conditions works better than trying to estimate using data


# Try using Cholesky Factorization

In [ ]:
# print(tf.linalg.det(H))
# H_sparse = tf.sparse.from_dense(H)
H_chol = tf.linalg.cholesky(H)
# print(H_chol)

In [ ]:
#getting weird numerical errors here making H very slightly non-symmetric
#  preventing me from using cholesky decomp.
H_ij = J[0].numpy().T @ tf.cast(omega[0], tf.float32) @ J[0]
# print(H_ij)
test = H_ij.numpy() - H_ij.numpy().T
# print(test)

print(H, "\n")
# test = np.tril(H.numpy())
test = np.tril(H.numpy(), -1)
# print(test)
print(np.tril(H.numpy()).T + np.tril(H.numpy(),-1) )

In [ ]:
# test = H.numpy() - H.numpy().T
h = np.random.randn(3,5)
ohm = np.eye(3)
test = h.T @ ohm @ h
print(test)

In [ ]:
#test
indices = tf.constant([[1,3], [3,1], [5,5]], dtype = tf.int64) #[N, ndims]
values = tf.constant([1.5, 1.5, -0.55])
dense_shape = tf.constant([6, 6], dtype = tf.int64)
sparse_J =  tf.sparse.SparseTensor(indices, values, dense_shape)
# print(sparse_J)
print(tf.sparse.to_dense(sparse_J))
# print(tf.linalg.det(tf.sparse.to_dense(sparse_J)))

test = tf.sparse.to_dense(sparse_J)[None,:,:] + 10*tf.eye(6)[None,:,:]
print(test)

chol = tf.linalg.cholesky(test) #needs to be symmetric and positive definite
print(chol)

rhs = tf.random.normal([1,6,6])
ans =  tf.linalg.cholesky_solve(chol, rhs)
print(ans)

Similar to ICET, we can use Newton-Raphson to iteratively solve for small perterbations to the state vector $x$ that bring us towards a better solution

\begin{equation}
\Large
x \rightarrow x + \Delta x
\end{equation}

# TODO

Figure out references in Jupyter Notebook


# Questions

Should $A_{ij} = -B_{ij}$?

Are there advantages to reporting state vector in unit quaternions? Besides being less intuitive to work with I've read they can also introduce problems in optimization routine since they add an additional degree of freedom. Every guide I've seen so far uses homogenous coordinate representation for the transformation matrices when computing the loss function, since they can be applied neatly in series. So rotation matrices do not produce singularities?

Is what we did in ICET to handle sparsity (i.e. accumulating contributions for corresponding elements rather than performing full matrix inversion) similar to a Choelesky Decomposition?

<span style="color:red"> What is going on with the numerics in get_e_ij()? Why is it not just t2v(Zij) - t2v(Zij)?? </span>



# Potential Contributions

Use ICET error covaraince estimation to demonstrate improvement in accuracy for Volpe Dataset

Use ICET output to inform $\Omega$ as a spatially dependant weighting field rather than simply indexing each unique point. Explore Gaussian Process Regression?


In [ ]:
test = tf.random.normal([600,600])
# print(tf.linalg.pinv(test))

https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.inv.html#scipy.sparse.linalg.inv

In [ ]:
st_a = tf.sparse.SparseTensor(indices=[[0, 2], [3, 4]],
                       values=[31.0, 2.0], 
                       dense_shape=[4, 6])
st_b = tf.sparse.SparseTensor(indices=[[0, 2], [3, 0]],
                       values=[56.0, 38.0],
                       dense_shape=[4, 6])
st_sum = tf.sparse.add(st_a, st_b)
# print(st_sum)
print(tf.linalg.pinv(tf.sparse.to_dense(st_sum)).numpy())